In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=207b7b54cd55246289b9d0081b4e4ff28d49054321e1886dec7adb6a0b11d219
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 4.4MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F16 '''
def F16(X):
    f = bn.F16()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.12 ,random_state=0 , l1_ratio=0.81, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_16_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_16_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(0, 10):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=277163, Fri Apr 17 20:32:45 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 -1.801812808110249e+02 1.0e+00 2.44e+00  2e+00  2e+00 0:00.2
termination on tolfun=1e-11 (Fri Apr 17 20:32:46 2020)
final/bestever f-value = -1.801813e+02 -1.801813e+02
incumbent solution: [ 1.77270073  0.26643304  3.97625855 -1.9682442   1.29671974 -0.36493665
 -4.16621303  0.25874513 ...]
std deviations: [2.4364075  2.43630874 2.43674096 2.43625807 2.43647874 2.43636993
 2.43637771 2.43654403 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=234890, Fri Apr 17 20:32:46 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 -1.801146458141778e+02 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.18128081] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.11464581] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


termination on tolfun=1e-11 (Fri Apr 17 20:32:46 2020)
final/bestever f-value = -1.801146e+02 -1.801146e+02
incumbent solution: [ 3.36232739 -0.36575409 -0.17018124 -2.46225781  2.17710328 -1.42633729
 -4.54280445  4.62471068 ...]
std deviations: [2.43608855 2.43598383 2.43615404 2.43581272 2.43622227 2.43590807
 2.43577486 2.43666592 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=187916, Fri Apr 17 20:32:46 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 -1.801999265907863e+02 1.0e+00 2.44e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.15553691] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 -1.802074887928315e+02 1.0e+00 2.38e+00  2e+00  2e+00 0:00.4


...

    3     57 -1.802076167846982e+02 1.0e+00 2.33e+00  2e+00  2e+00 0:00.3
   35    665 -1.802077547202393e+02 1.1e+00 1.94e+00  2e+00  2e+00 0:03.3
   78   1482 -1.802077394662919e+02 1.2e+00 1.91e+00  2e+00  2e+00 0:07.4
  100   1900 -1.802074038637171e+02 1.2e+00 1.93e+00  2e+00  2e+00 0:09.5
  163   3097 -1.802077204160077e+02 1.3e+00 1.87e+00  2e+00  2e+00 0:15.5
  200   3800 -1.802074428978201e+02 1.4e+00 2.07e+00  2e+00  2e+00 0:19.1
  285   5415 -1.802076658588090e+02 1.5e+00 1.89e+00  2e+00  2e+00 0:27.1
  300   5700 -1.802077782275482e+02 1.5e+00 1.95e+00  2e+00  2e+00 0:28.6
  400   7600 -1.802077800843312e+02 1.6e+00 1.90e+00  2e+00  2e+00 0:38.1
  500   9500 -1.802077794945234e+02 1.7e+00 1.86e+00  2e+00  2e+00 0:47.7
  600  11400 -1.802075952432793e+02 1.7e+00 2.10e+00  2e+00  2e+00 0:57.2
  700  13300 -1.802077642697840e+02 1.8e+00 1.95e+00  2e+00  2e+00 1:06.9
  800  15200 -1.802075843158888e+02 1.9e+00 1.75e+00  1e+00  2e+00 1:16.4
  900  17100 -1.802077135062161e+02 1.

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.18128081] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.11464581] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


termination on tolfun=1e-11 (Fri Apr 17 23:08:36 2020)
final/bestever f-value = -1.801146e+02 -1.801146e+02
incumbent solution: [ 1.07391566  1.27621242  3.04564589  0.73339001 -0.40378263  4.60573514
 -2.11080148 -2.63948811 ...]
std deviations: [2.43566523 2.43543535 2.43575843 2.43541415 2.43544967 2.43550895
 2.43548978 2.43566163 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=227948, Fri Apr 17 23:08:36 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 -1.802073019166714e+02 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 -1.802076074502581e+02 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.18330957] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.20279408] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 -1.802075274278954e+02 1.0e+00 2.33e+00  2e+00  2e+00 0:00.3
   35    665 -1.802073448787565e+02 1.1e+00 1.87e+00  2e+00  2e+00 0:03.3
   78   1482 -1.802077779388079e+02 1.2e+00 1.76e+00  2e+00  2e+00 0:07.4
  100   1900 -1.802077418322540e+02 1.2e+00 1.87e+00  2e+00  2e+00 0:09.5
  164   3116 -1.802077707024490e+02 1.4e+00 2.09e+00  2e+00  2e+00 0:15.6
  200   3800 -1.802077424641523e+02 1.4e+00 2.18e+00  2e+00  2e+00 0:19.0
  285   5415 -1.802077801872651e+02 1.5e+00 2.34e+00  2e+00  2e+00 0:27.0
  300   5700 -1.802074645400366e+02 1.6e+00 2.37e+00  2e+00  2e+00 0:28.4
  400   7600 -1.802075009800421e+02 1.6e+00 2.26e+00  2e+00  2e+00 0:38.0
  500   9500 -1.802075018011794e+02 1.7e+00 2.20e+00  2e+00  2e+00 0:47.4
  600  11400 -1.802077789800914e+02 1.7e+00 2.18e+00  2e+00  2e+00 0:56.9
  700  13300 -1.802076656733429e+02 1.8e+00 2.13e+00  2e+00  2e+00 1:06.4
  800  15200 -1.802077529454912e+02 1.8e+00 2.48e+00  2e+00  3e+00 1:16.0
  900  17100 -1.802077800500930e+02 1.

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F16_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F16(X_Values.iloc[i,:200])
    SVM_Fun [i] = F16(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F16(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([-188.01533179]), count=array([1])) 7.390929347522671
SVM
ModeResult(mode=array([-193.99471014]), count=array([1])) 9.491551953546054
ELN
ModeResult(mode=array([-188.37765365]), count=array([1])) 4.626397129991005
